<a href="https://colab.research.google.com/github/BangachevKiril/RepresentationLearningTheory/blob/plottingbranch/Copy_of_siglip_base_patch16_224.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U transformers

In [ ]:
device ='cuda'

Hugging face torch distribution of siglip: https://huggingface.co/collections/google/siglip-659d5e62f0ae1a57ae0e83ba

## Local Inference on GPU
Model page: https://huggingface.co/google/siglip-base-patch16-224

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/google/siglip-base-patch16-224)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

In [2]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("zero-shot-image-classification", model="google/siglip-base-patch16-224")
pipe(
    "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/hub/parrots.png",
    candidate_labels=["animals", "humans", "landscape"],
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use cuda:0


[{'score': 0.0007188215386122465, 'label': 'animals'},
 {'score': 4.1035607864614576e-05, 'label': 'humans'},
 {'score': 1.791011163732037e-05, 'label': 'landscape'}]

In [4]:
# Load model directly
from transformers import AutoProcessor, AutoModelForZeroShotImageClassification

processor = AutoProcessor.from_pretrained("google/siglip-base-patch16-224")
model = AutoModelForZeroShotImageClassification.from_pretrained("google/siglip-base-patch16-224")

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
resolution = 224

In [6]:
from PIL import Image
import requests
import torch
import torchvision.transforms as transforms

In [7]:
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

texts = ["a photo of 2 cats", "a photo of 2 dogs"]
inputs = processor(text=[''], images=image, padding="max_length", return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)

In [8]:
inputs['pixel_values'].shape

torch.Size([1, 3, 224, 224])

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !wget https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar

--2025-09-02 22:51:56--  https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar
Resolving image-net.org (image-net.org)... 171.64.68.16
Connecting to image-net.org (image-net.org)|171.64.68.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6744924160 (6.3G) [application/x-tar]
Saving to: ‘ILSVRC2012_img_val.tar’

ILSVRC2012_img_val. 100%[===================>]   6.28G  17.7MB/s    in 6m 12s  

2025-09-02 22:58:09 (17.3 MB/s) - ‘ILSVRC2012_img_val.tar’ saved [6744924160/6744924160]



In [10]:
from torchvision import transforms
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
val_transforms = transforms.Compose([
    transforms.Lambda(lambda image: image.convert('RGB')),
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

In [11]:
path = 'drive/MyDrive/imagenetdata/ImageNetValidation/Images/'

In [12]:
#!wget \https://raw.githubusercontent.com/tensorflow/models/master/research/slim/datasets/imagenet_2012_validation_synset_labels.txt

In [13]:
# !wget https://raw.githubusercontent.com/tensorflow/tpu/master/tools/datasets/imagenet_to_gcs.py

In [12]:
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

# Initialize ImageNet dataset
val_dataset = ImageFolder(root=path, transform=val_transforms)

# Optimize DataLoader settings
batch_size = 100
num_workers = 1
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,
                        num_workers=num_workers, pin_memory=True)

FileNotFoundError: Found no valid file for the classes drive. Supported extensions are: .jpg, .jpeg, .png, .ppm, .bmp, .pgm, .tif, .tiff, .webp

In [ ]:
import numpy as np
dim = 768
num = 50000
embeddings = np.zeros((num, dim))

In [ ]:
with torch.no_grad():
    for i, batch in enumerate(val_loader):
        inputs = processor(text=[''], images = batch[0], padding="max_length", return_tensors="pt")
        outputs = model(**inputs, do_rescale=False)
        embeddings[i*batch_size:(i+1)*batch_size, :] = outputs.image_embeds.cpu().numpy()
        print(i)
        if i == 2:
            break
        np.savez('drive/MyDrive/imagenetdata/torch_image_embeddings.npz', embeddings)

It looks like you are trying to rescale already rescaled images. If the input images have pixel values between 0 and 1, set `do_rescale=False` to avoid rescaling them again.


0
1


KeyboardInterrupt: 

In [20]:
# Preprocess val data

In [15]:
%ls drive/MyDrive/imagenetdata

Imagenet16_train_npz/     Imagenet64_validate/    ImageNetValidation/
Imagenet16_train_npz.zip  Imagenet64_val_npz.zip  torch_image_embeddings.npz


In [21]:
map = {}
with open('drive/MyDrive/imagenetdata/ImageNetValidation/map_clsloc.txt') as f:
    for line in f:
        line_split = line.split(' ')
        id = line_split[0]
        label = " ".join(line_split[-1].split('\n')[0].split('_'))
        map[int(line_split[1])] = label

In [24]:
import json
with open('drive/MyDrive/imagenetdata/ImageNetValidation/idx_to_name_map.json', 'w') as fp:
    json.dump(map, fp)

In [25]:
classes= []
map = {}
with open('drive/MyDrive/imagenetdata/ImageNetValidation/Labels.txt') as f:
    for line in f:
        classes.append(line.split('\n')[0])

In [26]:
classes[:100]

['490',
 '361',
 '171',
 '822',
 '297',
 '482',
 '13',
 '704',
 '599',
 '164',
 '649',
 '11',
 '73',
 '286',
 '554',
 '6',
 '648',
 '399',
 '749',
 '545',
 '13',
 '204',
 '318',
 '693',
 '399',
 '304',
 '102',
 '207',
 '480',
 '780',
 '644',
 '275',
 '14',
 '954',
 '249',
 '790',
 '501',
 '547',
 '809',
 '606',
 '297',
 '927',
 '424',
 '156',
 '60',
 '983',
 '256',
 '207',
 '281',
 '456',
 '413',
 '498',
 '561',
 '750',
 '182',
 '267',
 '118',
 '893',
 '597',
 '840',
 '836',
 '107',
 '647',
 '471',
 '945',
 '451',
 '214',
 '790',
 '291',
 '837',
 '707',
 '193',
 '397',
 '568',
 '401',
 '705',
 '200',
 '202',
 '31',
 '949',
 '361',
 '98',
 '709',
 '483',
 '563',
 '695',
 '122',
 '497',
 '914',
 '476',
 '102',
 '199',
 '104',
 '221',
 '138',
 '257',
 '188',
 '436',
 '229',
 '52']

In [27]:
with open('drive/MyDrive/imagenetdata/ImageNetValidation/val_class_split.txt','w') as fp:
  for i in range(50000):
    fp.write(f"ILSVRC2012_val_000"+"0"*(5 - len(str(i)))+str(i)+".JPEG "+classes[i]+'\n')